In [1]:
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import base64
import csv
import os
import seaborn as sns
import numpy as np
from glob import glob
from shutil import copyfile

%matplotlib inline

pd.set_option('display.max_columns', 100)

In [14]:
import os, keras
import numpy as np

from keras.preprocessing import image
from keras.utils.np_utils import to_categorical

from datetime import datetime
from matplotlib import pyplot as plt

from keras.models import Sequential
from keras.optimizers import SGD, RMSprop, Adam
from keras.layers import Lambda, Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D 
from keras.layers import Dropout, Flatten, merge, Reshape, Activation

%matplotlib inline

In [7]:
def vgg_preprocess(x):
    x = x - np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((3,1,1))
    return x[:, ::-1] # reverse axis rgb->bgr

In [8]:
def VGG16():
    
    """
    Model is based on 
    https://gist.github.com/baraldilorenzo/07d7802847aaad0a35d3
    ImageNet Pretrained Weights 
    https://drive.google.com/file/d/0Bz7KyqmuGsilT0J5dmRCM0ROVHc/view?usp=sharing

    """
    model = Sequential()
    model.add(Lambda(vgg_preprocess, input_shape=(3,224,224), output_shape=(3,224,224)))
    model.add(ZeroPadding2D((1,1),input_shape=(3,224,224)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation='softmax'))

    # Loads ImageNet pre-trained data
    model.load_weights('imagenet_weights/vgg16.h5')

    return model

In [2]:
PROJECT_DIR  = '/home/ubuntu/Extracting-food-preferences/'
PICTURES_DIR = '/home/ubuntu/Extracting-food-preferences/classification_with_pics/'

In [1]:
PROJECT_DIR  = 'C:\\Users\\Natalia\\Documents\\GitHub\\Extracting-food-preferences'
PICTURES_DIR = 'C:\\Users\\Natalia\\Documents\\GitHub\\Extracting-food-preferences\\classification_with_pics'

In [19]:
%cd $PROJECT_DIR

C:\Users\Natalia\Documents\GitHub\Extracting-food-preferences


In [4]:
from utils import *
from vgg16 import Vgg16
from datetime import datetime

%matplotlib inline

Using cuDNN version 5103 on context None
Mapped name None to device cuda: Tesla K80 (0000:00:1E.0)
Using Theano backend.


In [9]:
#%cd $PICTURES_DIR

path = PICTURES_DIR #+ '/sample/'
results_path=path + '/results/'
train_path=path + '/train/'
valid_path=path + '/valid/'

model_path = path + '/models/'
if not os.path.exists(model_path): os.mkdir(model_path)

C:\Users\Natalia\Documents\GitHub\Extracting-food-preferences\classification_with_pics


In [10]:
#%cd $PICTURES_DIR

path = PICTURES_DIR + '\\sample\\'
results_path=path + '\\results\\'
train_path=path + '\\train\\'
valid_path=path + '\\valid\\'

model_path = path + '\\models\\'
if not os.path.exists(model_path): os.mkdir(model_path)

C:\Users\Natalia\Documents\GitHub\Extracting-food-preferences\classification_with_pics


In [11]:
#Set constants
batch_size=25
no_of_epochs=1

In [15]:
generator = image.ImageDataGenerator()
val_batches = generator.flow_from_directory(valid_path, 
                                  target_size=(224,224), 
                                  class_mode='categorical',
                                  batch_size=batch_size, 
                                  shuffle=False)
batches     = generator.flow_from_directory(train_path, 
                                  target_size=(224,224), 
                                  class_mode='categorical',
                                  batch_size=batch_size, 
                                  shuffle=False)

Found 50 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [16]:
val_labels  = to_categorical(val_batches.classes)
trn_labels  = to_categorical(batches.classes)

In [7]:
batches = get_batches(path+'/train', shuffle=False, batch_size=batch_size)
val_batches = get_batches(path+'/valid', shuffle=False, batch_size=batch_size)

val_classes = val_batches.classes
trn_classes = batches.classes
val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)

Found 153087 images belonging to 2 classes.
Found 30000 images belonging to 2 classes.


In [17]:
batches.class_indices

{'popular': 0, 'unpopular': 1}

In [9]:
model = vgg_ft(2)

In [23]:
model = VGG16()

In [24]:
model.pop()
for layer in model.layers: layer.trainable=False
model.add(Dense(batches.nb_class, activation='softmax'))
model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy', metrics=['accuracy'])

In [41]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_2 (Lambda)                (None, 3, 224, 224)   0           lambda_input_2[0][0]             
____________________________________________________________________________________________________
zeropadding2d_14 (ZeroPadding2D) (None, 3, 226, 226)   0           lambda_2[0][0]                   
____________________________________________________________________________________________________
convolution2d_14 (Convolution2D) (None, 64, 224, 224)  1792        zeropadding2d_14[0][0]           
____________________________________________________________________________________________________
zeropadding2d_15 (ZeroPadding2D) (None, 64, 226, 226)  0           convolution2d_14[0][0]           
___________________________________________________________________________________________

In [26]:
model.load_weights(results_path+'ft1.h5')

In [43]:
layers = model.layers

In [45]:
last_conv_idx = [index for index,layer in enumerate(layers) 
                     if type(layer) is Convolution2D][-1]

In [48]:
conv_layers = layers[:last_conv_idx+1]
conv_model = Sequential(conv_layers)
fc_layers = layers[last_conv_idx+1:]

In [49]:
fc_layers

In [50]:
start = datetime.now()

val_features = conv_model.predict_generator(val_batches, val_batches.nb_sample)

end = datetime.now()
total = end - start
print('Time to run the script on GPU is {}'.format(total))

Time to run the script on GPU is 0:10:13.004283


In [51]:
save_array(model_path + 'valid_convlayer_features.bc', val_features)

In [56]:
start = datetime.now()

trn_features = conv_model.predict_generator(batches, batches.nb_sample)

end = datetime.now()
total = end - start
print('Time to run the script on GPU is {}'.format(total))

MemoryError: 

In [65]:
fname = model_path + 'train_convlayer_features.bc'
for i in xrange(batches.n // batch_size + 1):
    conv_feat = conv_model.predict_on_batch(batches.next()[0])
    if not i:
        c = bcolz.carray(conv_feat, rootdir=fname, mode='a')
    else:
        c.append(conv_feat)
c.flush()

In [25]:
#save_array(model_path + 'train_convlayer_features.bc', trn_features)

In [68]:
#trn_features = load_array(model_path+'train_convlayer_features.bc') #loading trainset causes memory error
val_features = load_array(model_path+'valid_convlayer_features.bc')

In [69]:
val_features.shape

(30000, 512, 14, 14)

In [70]:
# Such a finely tuned model needs to be updated very slowly!
opt = RMSprop(lr=0.001, rho=0.7)

In [71]:
def proc_wgts(layer): return [o/2 for o in layer.get_weights()]
def get_fc_model():
    model = Sequential([
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.),
        Dense(4096, activation='relu'),
        Dropout(0.),
        Dense(2, activation='softmax')
        ])

    for l1,l2 in zip(model.layers, fc_layers): l1.set_weights(proc_wgts(l2))

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [72]:
fc_model = get_fc_model()

In [31]:
fc_model.fit(trn_features, trn_labels, nb_epoch=1, 
             batch_size=batch_size, validation_data=(val_features, val_labels))

Train on 200 samples, validate on 50 samples
Epoch 1/1
200/200 [==============================] - 0s - loss: 8.0590 - acc: 0.5000 - val_loss: 6.4472 - val_acc: 0.6000


In [32]:
fc_model.save_weights(model_path+'no_dropout.h5')